In [1]:
from transformers import Trainer, BertForSequenceClassification, BertTokenizer, EarlyStoppingCallback, AutoConfig, TrainingArguments, AutoModelForImageClassification
from datasets import load_from_disk
from torch.utils.data import DataLoader
import torch
from torch import nn
from torch.nn import functional as F
import base
import os 

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [2]:
dataset_part = base.get_dataset_part()
DATASET = "cifar100"

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 1g.10gb


In [4]:
transform = base.base_transforms()

train = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.TRAIN, transform=transform)
eval = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.EVAL, transform=transform)
test = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.TEST, transform=transform)

In [5]:
augment_transform = base.aug_transforms()

train_aug = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.TRAIN, transform=augment_transform)

In [48]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/vit-basetrain", logging_dir=f"~/logs/{DATASET}/vit-basetrain", lr=0.0005, weight_decay=0.008, adam_beta1=.95, epochs=30)
student_model = AutoModelForImageClassification.from_pretrained("timm/tiny_vit_5m_224.in1k", num_labels=100, ignore_mismatched_sizes=True)

Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([100]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([100, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [64]:
print(model)

TinyVit(
  (patch_embed): PatchEmbed(
    (conv1): ConvNorm(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (act): GELU(approximate='none')
    (conv2): ConvNorm(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (stages): Sequential(
    (0): ConvLayer(
      (blocks): Sequential(
        (0): MBConv(
          (conv1): ConvNorm(
            (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (act1): GELU(approximate='none')
          (conv2): ConvNorm(
            (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=256, bias=Fals

In [ ]:
model = timm.create_model("tiny_vit_5m_224.in1k", pretrained=True)
hidden_states = []

def hook_fn(module, input, output):
    hidden_states.append(output)

# Example: assuming the model has an attribute 'stages' which is a list of modules

    # Depending on the structure, you might need to iterate deeper.
    # For example, if each stage is a sequential container of blocks:
for block in model.stages[3].children():
    
    print(block)
    block.register_forward_hook(hook_fn)

dummy_input = torch.randn(1, 3, 224, 224)
_ = model(dummy_input)
print(f"Captured {len(hidden_states)} hidden states.")  
print(hidden_states[1].shape)


PatchMerging(
  (conv1): ConvNorm(
    (conv): Conv2d(160, 320, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(320, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (act1): GELU(approximate='none')
  (conv2): ConvNorm(
    (conv): Conv2d(320, 320, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=320, bias=False)
    (bn): BatchNorm2d(320, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (act2): GELU(approximate='none')
  (conv3): ConvNorm(
    (conv): Conv2d(320, 320, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(320, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)
Sequential(
  (0): TinyVitBlock(
    dim=320, num_heads=10, window_size=7, mlp_ratio=4.0
    (attn): Attention(
      (norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
      (qkv): Linear(in_features=320, out_features=960, bias=True)
      (proj): Linear(in_features=320, out_features=320, bias=Tru

In [68]:
hidden_states = {}
model = timm.create_model("tiny_vit_5m_224.in1k", pretrained=True)
def get_hook(name):
    def hook(module, input, output):
        hidden_states[name] = output
        print(f"Captured hidden state from '{name}' with shape {output.shape}")
    return hook

# Suppose you’ve identified a list of candidate modules (for example, from model.stages or similar)
for name, module in model.named_modules():
    # Replace the condition with the proper filter for your architecture
    if 'stage' in name or 'block' in name:  # adjust as needed for TinyViT's naming conventions
        module.register_forward_hook(get_hook(name))

In [69]:
dummy_input = torch.randn(1, 3, 224, 224)
_ = model(dummy_input)
print(f"Captured {len(hidden_states)} hidden states.")  
print(hidden_states)

Captured hidden state from 'stages.0.blocks.0.conv1.conv' with shape torch.Size([1, 256, 56, 56])
Captured hidden state from 'stages.0.blocks.0.conv1.bn' with shape torch.Size([1, 256, 56, 56])
Captured hidden state from 'stages.0.blocks.0.conv1' with shape torch.Size([1, 256, 56, 56])
Captured hidden state from 'stages.0.blocks.0.act1' with shape torch.Size([1, 256, 56, 56])
Captured hidden state from 'stages.0.blocks.0.conv2.conv' with shape torch.Size([1, 256, 56, 56])
Captured hidden state from 'stages.0.blocks.0.conv2.bn' with shape torch.Size([1, 256, 56, 56])
Captured hidden state from 'stages.0.blocks.0.conv2' with shape torch.Size([1, 256, 56, 56])
Captured hidden state from 'stages.0.blocks.0.act2' with shape torch.Size([1, 256, 56, 56])
Captured hidden state from 'stages.0.blocks.0.conv3.conv' with shape torch.Size([1, 64, 56, 56])
Captured hidden state from 'stages.0.blocks.0.conv3.bn' with shape torch.Size([1, 64, 56, 56])
Captured hidden state from 'stages.0.blocks.0.conv

In [49]:
teacher_model = AutoModelForImageClassification.from_pretrained(
    "Ahmed9275/Vit-Cifar100",
    num_labels=100,
)

teacher_model.to(device)
teacher_model.eval()

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fe

In [78]:
class DistilTrainerInner(Trainer):
    """Distilation trainer, computes loss with logits from teacher in mind. Logits are precomputed."""
    def __init__(self, student_model=None, teacher_model = None, *args, **kwargs):
        super().__init__(model=student_model, *args, **kwargs)
        self.student = student_model
        self.teacher = teacher_model
        self.layer_loss_function = nn.MSELoss()
        self.logit_loss_function = nn.KLDivLoss(reduction="batchmean")
        self.temperature = self.args.temperature
        self.lambda_param = self.args.lambda_param
        self.alpha_param = self.args.alpha_param


        self.student_to_teacher = nn.Linear(128, 768).to(device)
        self.model_parameters = list(self.model.parameters()) + list(self.student_to_teacher.parameters())

    def compute_loss(self, student, inputs, return_outputs=False, num_items_in_batch=None):
        logits = inputs.pop("logits")
        with torch.no_grad():
            teacher_output = self.teacher(**inputs, output_hidden_states=True)
        teacher_hidden_states = teacher_output.hidden_states
        print(teacher_hidden_states[6].shape)
        student_output = student(**inputs, output_hidden_states=True)
        student_target_loss = student_output["loss"]
        
        
        

        teacher_hidden_states = teacher_output.hidden_states
        student_hidden_states = student_output.hidden_states
        print(student_hidden_states)

        teacher_l6 = teacher_hidden_states[6] / self.temperature
        teacher_l12 = teacher_hidden_states[12] / self.temperature
        
        student_l1 = student_hidden_states[1]
        student_l2 = student_hidden_states[2] 

        student_l1_projection = self.student_to_teacher(student_l1) / self.temperature
        student_l2_projection = self.student_to_teacher(student_l2) / self.temperature

        layer_distillation_loss = (
            self.layer_loss_function(student_l1_projection, teacher_l6) +
            self.layer_loss_function(student_l2_projection, teacher_l12)
        )

        

        soft_teacher = F.softmax(logits / self.temperature, dim=-1)
        soft_student = F.log_softmax(student_output['logits'] / self.temperature, dim=-1)

        logit_distillation_loss = self.logit_loss_function(soft_student, soft_teacher) * (self.temperature ** 2)
        logit_label_loss = ((1. - self.lambda_param) * student_target_loss + self.lambda_param * logit_distillation_loss)

        
        loss = (1 - self.alpha_param) * logit_label_loss + self.alpha_param * layer_distillation_loss

        
        return (loss, student_output) if return_outputs else loss
    
    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        logits = inputs.pop("logits")
        inputs = self._prepare_inputs(inputs)
        with torch.no_grad():
            # For evaluation, we disable extra outputs.
            outputs = model(**inputs, output_hidden_states=False)
            loss = outputs.loss if "loss" in outputs else None
            logits = outputs.logits
        labels = inputs.get("labels")
        return loss, logits, labels

In [79]:
class Custom_training_args(TrainingArguments):
    """Custom wrapper of training args for distillation."""
    def __init__(self, lambda_param, alpha_param, temperature, *args, **kwargs):
        super().__init__(*args, **kwargs)    
        self.lambda_param = lambda_param
        self.alpha_param = alpha_param
        self.temperature = temperature

In [80]:
def get_training_args(output_dir, logging_dir, remove_unused_columns=True, lr=5e-5, epochs=5, weight_decay=0, adam_beta1 = .9, lambda_param=.5, alpha_param = .5, temp=5, batch_size=128, num_workers=4, warmup_steps=0):
    """Returns training args that can be adjusted."""
    return (
        Custom_training_args(
        output_dir=output_dir,
        eval_strategy="epoch",
        adam_beta1 = adam_beta1,
        warmup_steps = warmup_steps,
        save_strategy="epoch",
        logging_strategy="epoch",
        learning_rate=lr, #Defaultní hodnota 
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        weight_decay=weight_decay,
        seed = 42,  #Defaultní hodnota 
        metric_for_best_model="f1",
        load_best_model_at_end = True,
        fp16=True, 
        logging_dir=logging_dir,
        remove_unused_columns=remove_unused_columns,
        lambda_param = lambda_param,
        alpha_param = alpha_param, 
        temperature = temp,
        dataloader_num_workers=num_workers,
        )
    )

In [81]:
training_args = get_training_args(output_dir=f"~/results/{DATASET}/hokus_pokus", logging_dir=f"~/logs/{DATASET}/hokus_pokus", remove_unused_columns=False, warmup_steps=4, lr=5e-4, weight_decay=.003, batch_size=128, epochs=20, temp=2.5, lambda_param=.4, alpha_param=.5)

In [82]:
trainer = DistilTrainerInner(
    student_model = student_model,
    teacher_model = teacher_model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [83]:
trainer.train()

torch.Size([128, 197, 768])


ValueError: The 'output_hidden_states' option cannot be set for this timm model. To enable this feature, the 'forward_intermediates' method must be implemented in the timm model (available in timm versions > 1.*). Please consider using a different architecture or updating the timm package to a compatible version.

In [24]:
student_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [25]:
trainer.evaluate(test)

{'eval_loss': 1.0279921293258667,
 'eval_accuracy': 0.76,
 'eval_precision': 0.5204448048364814,
 'eval_recall': 0.5517825303517305,
 'eval_f1': 0.5117426131149784,
 'eval_runtime': 3.1235,
 'eval_samples_per_second': 160.077,
 'eval_steps_per_second': 1.281,
 'epoch': 20.0}

In [26]:
base.reset_seed()

In [27]:
student_model = BertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2", num_labels=50)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
training_args = get_training_args(output_dir=f"~/results/{DATASET}/hokus_pokus", logging_dir=f"~/logs/{DATASET}/hokus_pokus", remove_unused_columns=False, warmup_steps=4, lr=5e-4, weight_decay=.003, batch_size=128, epochs=20, temp=2.5, lambda_param=.4, alpha_param=.5)

In [ ]:
class DistilTrainerInnerAVG(Trainer):
    """Distilation trainer, computes loss with logits from teacher in mind. Logits are precomputed."""
    def __init__(self, student_model=None, teacher_model = None, *args, **kwargs):
        super().__init__(model=student_model, *args, **kwargs)
        self.student = student_model
        self.teacher = teacher_model
        self.layer_loss_function = nn.MSELoss()
        self.logit_loss_function = nn.KLDivLoss(reduction="batchmean")
        self.temperature = self.args.temperature
        self.lambda_param = self.args.lambda_param
        self.alpha_param = self.args.alpha_param


        self.student_to_teacher = nn.Linear(128, 768).to(device)
        self.model_parameters = list(self.model.parameters()) + list(self.student_to_teacher.parameters())

    def compute_loss(self, student, inputs, return_outputs=False, num_items_in_batch=None):
        logits = inputs.pop("logits")
        
        student_output = student(**inputs, output_hidden_states=True)
        student_target_loss = student_output["loss"]

        with torch.no_grad():
            teacher_output = self.teacher(**inputs, output_hidden_states=True)

        teacher_hidden_states = teacher_output.hidden_states
        student_hidden_states = student_output.hidden_states

        
        
        teacher_l6 = torch.stack(teacher_hidden_states[2:7], dim=0).mean(dim=0) / self.temperature
        teacher_l12 = torch.stack(teacher_hidden_states[7:12], dim=0).mean(dim=0) / self.temperature
        student_l1 = student_hidden_states[1]
        student_l2 = student_hidden_states[2] 

        student_l1_projection = self.student_to_teacher(student_l1) / self.temperature
        student_l2_projection = self.student_to_teacher(student_l2) / self.temperature

        layer_distillation_loss = (
            self.layer_loss_function(student_l1_projection, teacher_l6) +
            self.layer_loss_function(student_l2_projection, teacher_l12)
        )

        

        soft_teacher = F.softmax(logits / self.temperature, dim=-1)
        soft_student = F.log_softmax(student_output['logits'] / self.temperature, dim=-1)

        logit_distillation_loss = self.logit_loss_function(soft_student, soft_teacher) * (self.temperature ** 2)
        logit_label_loss = ((1. - self.lambda_param) * student_target_loss + self.lambda_param * logit_distillation_loss)

        
        loss = (1 - self.alpha_param) * logit_label_loss + self.alpha_param * layer_distillation_loss

        
        return (loss, student_output) if return_outputs else loss
    
    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        logits = inputs.pop("logits")
        inputs = self._prepare_inputs(inputs)
        with torch.no_grad():
            # For evaluation, we disable extra outputs.
            outputs = model(**inputs, output_hidden_states=False)
            loss = outputs.loss if "loss" in outputs else None
            logits = outputs.logits
        labels = inputs.get("labels")
        return loss, logits, labels

In [33]:
trainer = DistilTrainerInnerAVG(
    student_model = student_model,
    teacher_model = teacher_model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [34]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.480100,2.863770,0.418882,0.087182,0.101284,0.076397
2,1.103200,2.138775,0.562786,0.208789,0.197205,0.174814
3,0.821700,1.635681,0.655362,0.267159,0.258225,0.237654
4,0.631500,1.374435,0.713107,0.302741,0.315566,0.295314
5,0.510500,1.259040,0.728689,0.369574,0.356522,0.335380
6,0.430800,1.177486,0.739688,0.364386,0.368775,0.350246
7,0.369200,1.139008,0.757104,0.475266,0.424130,0.416781
8,0.332200,1.116509,0.758020,0.405699,0.433149,0.410593
9,0.294900,1.078361,0.754354,0.425039,0.422598,0.410365
10,0.266500,1.060170,0.764436,0.461547,0.435401,0.431252


TrainOutput(global_step=700, training_loss=0.4159424332209996, metrics={'train_runtime': 801.2602, 'train_samples_per_second': 108.854, 'train_steps_per_second': 0.874, 'total_flos': 65900954952000.0, 'train_loss': 0.4159424332209996, 'epoch': 20.0})